## Устанавливаем библиотеки и загружаем данные

In [1]:
!pip install razdel
import gensim
import numpy as np
from nltk.tokenize import sent_tokenize
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from collections import Counter
import nltk
nltk.download('punkt')

!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/2ch_corpus.txt.zip  -O dvach_corpus.txt.zip
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/lenta.txt.zip -O lenta.txt.zip

!unzip dvach_corpus.txt.zip
!unzip lenta.txt.zip

lenta = 'lenta.txt'
dvach = '2ch_corpus.txt'
with open(lenta, 'r', encoding='utf-8') as f:
    lenta = f.read()

with open(dvach, 'r', encoding='utf-8') as f:
    dvach = f.read()

lenta = lenta[:50000]
dvach = dvach[:50000]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
--2021-06-23 23:07:40--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/2ch_corpus.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6255552 (6.0M) [application/zip]
Saving to: ‘dvach_corpus.txt.zip’

dvach_corpus.txt.zi 100%[===================>]   5.96M  --.-KB/s    in 0.06s   

2021-06-23 23:07:40 (101 MB/s) - ‘dvach_corpus.txt.zip’ saved [6255552/6255552]

--2021-06-23 23:07:40--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/lenta.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw

# Задание 1

In [2]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

def create_ngrams(sentences):
    unigrams, bigrams, trigrams = Counter(), Counter(), Counter()
    for sentence in sentences:
        unigrams.update(sentence)
        bigrams.update(ngrammer(sentence, n=2))
        trigrams.update(ngrammer(sentence, n=3))
    return unigrams, bigrams, trigrams

In [3]:
sentences_dvach = [ ['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [ ['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(lenta)]

unigrams_dvach, bigrams_dvach, trigrams_dvach = create_ngrams(sentences_dvach)
unigrams_news, bigrams_news, trigrams_news = create_ngrams(sentences_news)

In [4]:
def create_matrix(unigrams, bigrams, trigrams):
    matrix_trigrams = np.zeros((len(bigrams), len(unigrams)))
    matrix_bigrams = np.zeros((len(unigrams), len(unigrams)))
    id2bigram = list(bigrams)
    bigram2id  =  {bigram:i for i, bigram in enumerate(id2bigram)}
    id2word = list(unigrams)
    word2id = {word:i for i, word in enumerate(id2word)}
    for ngram in trigrams:
        ngram_ = ngram.split(' ')
        bigram, unigram = ' '.join(ngram_[:-1]), ngram_[-1]
        matrix_trigrams[bigram2id[bigram]][word2id[unigram]] =  (trigrams[ngram] / bigrams[bigram])
    for ngram in bigrams:
        word1, word2 = ngram.split(' ')
        matrix_bigrams[word2id[word1]][word2id[word2]] =  (bigrams[ngram] / unigrams[word1])
    return matrix_trigrams, matrix_bigrams, id2bigram, bigram2id, id2word, word2id

In [5]:
matrix_trigrams_dvach, matrix_bigrams_dvach, id2bigram_dvach, bigram2id_dvach, id2word_dvach, word2id_dvach = create_matrix(unigrams_dvach, bigrams_dvach, trigrams_dvach)

matrix_trigrams_news, matrix_bigrams_news, id2bigram_news, bigram2id_news, id2word_news, word2id_news = create_matrix(unigrams_news, bigrams_news, trigrams_news)

In [6]:
def bigrams_generator(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        if id2word[chosen] == '<end>':
            chosen = word2id[start]
        current_idx = chosen
    return ' '.join(text)

def trigrams_generator(matrix, id2word, ngram2id, id2ngram, n=100, start='<start> <start>'):
    text = []
    current_idx = ngram2id[start]
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        if id2word[chosen] == '<end>':
            token = start
        else:
            token = id2ngram[current_idx].split()[-1] + " " + id2word[chosen]
        current_idx = ngram2id[token]
    return ' '.join(text)

In [7]:
def comparison(matrix_bigrams, matrix_trigrams, id2word, word2id, bigram2id, id2bigram, n=100):
    print('Bigram-based Texts')
    print(bigrams_generator(matrix=matrix_bigrams, id2word=id2word, word2id=word2id).replace('<end>', '\n'))
    print()
    print('Trigram-based Texts')
    print(trigrams_generator(matrix=matrix_trigrams, id2word=id2word, ngram2id=bigram2id, id2ngram=id2bigram).replace('<end>', '\n'))

In [8]:
# Генерируем Двач
comparison(matrix_bigrams_dvach, matrix_trigrams_dvach, id2word_dvach, word2id_dvach, bigram2id_dvach, id2bigram_dvach)

Bigram-based Texts
у меня тут прилагательное и если не слышал 
 мне тоже но не нужны совсем зелёный и только выйдет еще очень заметно 
 <start> scala kotlin swift go и зашоренность приводит 
 <start> соус ремикса с другой сами deen остальное ml семейство энергичное energetic evaluation чёт не юно 
 просто тянок было лишь исключением что надо указывать качество q a параметр vbr вместо битрейта b a работает только 35 slowfix суть в чем переучить какого-нибудь джявадауна с гитхаба никто 
 диванный кукаретик сбежал роняя кал чтд то шарпер ide а там не слышал или на осунувшееся лицо товарища стало серии

Trigram-based Texts
и гугл сделает из двух лямов индусов лучших девелоперов чем mit пок-пок не выёбуйся жри говно большой ложкой пока не придёт разнарядка от директора аутсорс-галеры остальная меньшая часть начинает бездумно сверкать новым базвордом в бложиках и пихать модное говно куда попало 
 говоришь так будто бы это не так 
 даже если согласится что это толстый тралленг пожалуйста 
 ло

In [10]:
# Генерируем Новости
comparison(matrix_bigrams_news, matrix_trigrams_news, id2word_news, word2id_news, bigram2id_news, id2bigram_news)

Bigram-based Texts
<start> в три человека 27 было заключено 2011 сделок на сумму в зону евро учитывая адаптацию российских банков в хатах 
 <start> <start> <start> <start> <start> <start> не использовать для участия в литве этот срок от доргели в день церемония вступления в кресле 
 <start> он в тяжелом состоянии 
 <start> <start> <start> россия 
 <start> <start> сегодняшнее сообщение о разморозке кредита показывает что этот аудит не сообщил сегодня во время там на минувшей ночью была выведена из толпы раздались крики и лекарства радиоактивные материалы объекты флоры и второго пришествия христа 
 подразделения министр обороны заявило во фронт 
 <start>

Trigram-based Texts
представитель посольства сша в китайском городе ченду 
 иллюстрированный журнал « искры » № 37 сентябрь 1914 daily mirror рассказывает случай который порадует всех друзей животных 
 попытка германской пехоты пробиться ближе к концу сентября а не в середине сентября этот срок более реален с организационной точки зрени

# Задание 2

In [11]:
with open('lenta.txt', 'r', encoding='utf-8') as f:
    lenta = f.read()

with open('2ch_corpus.txt', 'r', encoding='utf-8') as f:
    dvach = f.read()
    
sentences_dvach = [normalize(text)for text in sent_tokenize(dvach)]
sentences_news = [normalize(text)for text in sent_tokenize(lenta)]

In [12]:
def pmi_score(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
    if bigram_count >= min_count:
        try:
            prob_a = worda_count / corpus_word_count
            prob_b = wordb_count / corpus_word_count
            prob_ab = bigram_count / corpus_word_count
            pmi = np.log(prob_ab / (prob_a * prob_b))
        except ZeroDivisionError:
            pmi = float('-inf') 
    else:
        pmi = float('-inf') 
    return pmi

In [13]:
ph_default = gensim.models.Phrases(sentences_dvach, threshold=0.0001)
p_default = gensim.models.phrases.Phraser(ph_default)

ph2_default = gensim.models.Phrases(p_default[sentences_dvach])
p2_default = gensim.models.phrases.Phraser(ph2_default)
for i in range(10):
    print(p2_default[p_default[sentences_dvach[i]]])

['анимублядский_webm-треддля_приличных_анимублядей', 'и_прочих_аутистов']
['безграмотное_быдло_с_дубляжом', 'войсовером_порнографией_и_котиками', 'советы_мерзких_мокрописечников_вниманиебляди', 'всех_видов_и_прочее', 'непотребство_отправляется_в_порнотред', 'ссылка', 'для_поиска_с_o', 'уса_видео_сохраняем_кадр', 'правый_клик_по_видео', 'и_ищем_его_на', 'для_воспроизведения_webm_с', '10-битным_цветом_нужно_установить', 'плагин_vlc_и_отключить', 'встроенный_в_браузер_плеер', 'media']
['webm']
['enabled_false_в_firefox', 'о_кодировании_webm_доступные', 'кодеки_—_vp_8', 'и_vp_9_для', 'видео_vorbis_и_opus', 'для_звука_максимальный_размер', 'файла_—', '10240', 'кб_всех_файлов_в', 'посте_—', 'около_40', 'мб']
['делать_webm_можно_научиться', 'в_вики_треда_там', 'находится_подробная_информация_о', 'выборе_и_настройке_кодеков', 'на_примерах_использования_консольных', 'утилит_ffmpeg_vpxenc_и', 'mkvmerge']
['неочевидные_моменты—_libvorbis_при', 'указании_битрейта_b_a', 'работает_в_режиме_cbr', 'по

In [14]:
ph_custom = gensim.models.Phrases(sentences_dvach, scoring=pmi_score, threshold=0.0001)
p_custom = gensim.models.phrases.Phraser(ph_custom)

ph2_custom = gensim.models.Phrases(p_custom[sentences_dvach], scoring=pmi_score)
p2_custom = gensim.models.phrases.Phraser(ph2_custom)
for i in range(10):
    print(p2_custom[p_custom[sentences_dvach[i]]])

['анимублядский_webm-треддля_приличных_анимублядей', 'и_прочих', 'аутистов']
['безграмотное_быдло_с_дубляжом', 'войсовером_порнографией_и_котиками', 'советы_мерзких_мокрописечников_вниманиебляди', 'всех_видов', 'и_прочее', 'непотребство_отправляется_в_порнотред', 'ссылка', 'для_поиска_с_o', 'уса_видео_сохраняем_кадр', 'правый_клик_по_видео', 'и_ищем', 'его_на', 'для_воспроизведения_webm_с', '10-битным_цветом_нужно_установить', 'плагин_vlc_и_отключить', 'встроенный_в_браузер_плеер', 'media']
['webm']
['enabled_false_в_firefox', 'о_кодировании_webm_доступные', 'кодеки_—_vp_8', 'и_vp_9_для', 'видео_vorbis_и_opus', 'для_звука_максимальный_размер', 'файла_—', '10240', 'кб_всех_файлов_в', 'посте_—', 'около_40', 'мб']
['делать_webm_можно_научиться', 'в_вики_треда_там', 'находится_подробная_информация_о', 'выборе_и_настройке_кодеков', 'на_примерах_использования_консольных', 'утилит_ffmpeg_vpxenc_и', 'mkvmerge']
['неочевидные_моменты—_libvorbis_при', 'указании_битрейта_b_a', 'работает_в_режиме_

In [15]:
ph_default = gensim.models.Phrases(sentences_dvach, threshold=13)
p_default = gensim.models.phrases.Phraser(ph_default)

ph2_default = gensim.models.Phrases(p_default[sentences_dvach])
p2_default = gensim.models.phrases.Phraser(ph2_default)
for i in range(10):
    print(p2_default[p_default[sentences_dvach[i]]])

['анимублядский_webm-треддля_приличных_анимублядей', 'и_прочих_аутистов']
['безграмотное_быдло_с_дубляжом', 'войсовером_порнографией', 'и', 'котиками_советы_мерзких_мокрописечников', 'вниманиебляди_всех_видов', 'и_прочее_непотребство_отправляется', 'в_порнотред', 'ссылка', 'для_поиска_с', 'o_уса_видео_сохраняем', 'кадр_правый_клик_по', 'видео', 'и', 'ищем_его_на', 'для_воспроизведения_webm', 'с_10-битным_цветом_нужно', 'установить_плагин_vlc', 'и', 'отключить_встроенный', 'в', 'браузер_плеер_media']
['webm']
['enabled_false_в_firefox', 'о_кодировании_webm_доступные', 'кодеки_—_vp_8', 'и', 'vp_9_для', 'видео_vorbis', 'и', 'opus_для_звука_максимальный', 'размер_файла_—', '10240', 'кб_всех_файлов', 'в', 'посте', '—_около', '40_мб']
['делать_webm', 'можно_научиться', 'в', 'вики_треда_там', 'находится_подробная_информация_о', 'выборе', 'и', 'настройке_кодеков_на', 'примерах_использования_консольных_утилит', 'ffmpeg_vpxenc', 'и', 'mkvmerge']
['неочевидные_моменты—_libvorbis_при', 'указании_б

In [16]:
ph_custom = gensim.models.Phrases(sentences_dvach, scoring=pmi_score, threshold=13)
p_custom = gensim.models.phrases.Phraser(ph_custom)

ph2_custom = gensim.models.Phrases(p_custom[sentences_dvach], scoring=pmi_score)
p2_custom = gensim.models.phrases.Phraser(ph2_custom)
for i in range(10):
    print(p2_custom[p_custom[sentences_dvach[i]]])

['анимублядский_webm-треддля', 'приличных_анимублядей', 'и', 'прочих', 'аутистов']
['безграмотное', 'быдло', 'с', 'дубляжом_войсовером', 'порнографией', 'и', 'котиками', 'советы', 'мерзких_мокрописечников', 'вниманиебляди', 'всех', 'видов', 'и', 'прочее', 'непотребство_отправляется', 'в', 'порнотред', 'ссылка', 'для', 'поиска', 'с', 'o', 'уса', 'видео', 'сохраняем_кадр', 'правый_клик', 'по', 'видео', 'и', 'ищем', 'его', 'на', 'для', 'воспроизведения', 'webm', 'с', '10-битным_цветом', 'нужно', 'установить', 'плагин_vlc', 'и', 'отключить_встроенный', 'в', 'браузер', 'плеер_media']
['webm']
['enabled_false', 'в', 'firefox', 'о', 'кодировании', 'webm', 'доступные_кодеки', '—', 'vp', '8', 'и', 'vp', '9', 'для', 'видео', 'vorbis', 'и', 'opus', 'для', 'звука', 'максимальный', 'размер', 'файла', '—', '10240', 'кб', 'всех', 'файлов', 'в', 'посте', '—', 'около', '40', 'мб']
['делать', 'webm', 'можно', 'научиться', 'в', 'вики', 'треда', 'там', 'находится', 'подробная_информация', 'о', 'выборе', '

Works pretty similar.